In [ ]:
!pip install groq

In [ ]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ-llama-3.1')

In [ ]:
import requests
import json

input_file_path = "sample_data/original_dataset.json"
output_file_path = "sample_data/llama_3.1_output.json"

# input_file_path = "sample_data/test_original.json"
# output_file_path = "sample_data/test_output.json"

In [ ]:
with open(input_file_path, "r") as input_file:
    input_data = json.load(input_file)

In [ ]:
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

base_instruction = """ When given the below JSON formatted file content, I need you to give me the specific sentences from the text that exhibit a set of symptoms which are under the annotations section. Below is an example of INPUT and OUTPUT. Just give me the outputs, do not give any code or re-output the prompting, do not just give "yes" or "no" again, give the actual sentences. Keep JSON Formatting. And make sure to always return all the components of each of the objects, so always include "post_text", "post_title", and "annotations":

{
    "post_title": "I don't feel original anymore.",
    "post_text": "When I was in high school a few years back, I was one of the highest competitors in my school. I joined the high school band in freshman year and by senior year I became one of the best in my section. My academics were always straight and I exercised daily. Senior year I enlisted in the military and now I believe it was one of my worst decisions in life. Before I went to boot camp I was motivated, a patriot and believed that the elite joined the military. In senior year I never applied for any scholarships and I was offered one but turned it down because I already signed the papers. I thought I set myself up for success. Now I believe I was dead wrong for joining. The only benefit I see so far after a year and a half of service is that I'm trying to set myself up financially before I get out and hopefully attend college. It sounds like a plan but I feel no happiness from what I do at all. I convinced myself there's no honor in it anymore, it's just another job. I don't exercise by myself anymore. I feel like I'm not progressing anywhere in life being in service. I'm just a body and if I wasn't here doing what I'm doing, there'd just be somebody else doing the exact same. I'm replaceable. That's the mindset the military gave me. I look forward to going back home in 6 months for vacation and that's the only thing I've been looking forward to since I've been stationed. After that, the only thing I have my eyes on are getting out of service, going home, being closer to my family again. There's nothing here that satisfies me and I hate it. I feel like I've tried everything to be happy here but it seems impossible. I wish somebody could help.",
    "annotations": [
      [
        "Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down",
        "yes"
      ],
      [
        "Feeling-down-depressed-or-hopeless",
        "no"
      ],
      [
        "Feeling-tired-or-having-little-energy",
        "yes"
      ],
      [
        "Little-interest-or-pleasure-in-doing",
        "yes"
      ],
      [
        "Moving-or-speaking-so-slowly-that-other-people-could-have-noticed-Or-the-opposite-being-so-fidgety-or-restless-that-you-have-been-moving-around-a-lot-more-than-usual",
        "no"
      ],
      [
        "Poor-appetite-or-overeating",
        "no"
      ],
      [
        "Thoughts-that-you-would-be-better-off-dead-or-of-hurting-yourself-in-some-way",
        "no"
      ],
      [
        "Trouble-concentrating-on-things-such-as-reading-the-newspaper-or-watching-television",
        "no"
      ],
      [
        "Trouble-falling-or-staying-asleep-or-sleeping-too-much",
        "no"
      ]
    ]
}

And this is an example expected output format:

{
    "post_title": "I don't feel original anymore.",
    "post_text": "When I was in high school a few years back, I was one of the highest competitors in my school. I joined the high school band in freshman year and by senior year I became one of the best in my section. My academics were always straight, and I exercised daily. Senior year I enlisted in the military, and now I believe it was one of my worst decisions in life. Before I went to boot camp I was motivated, a patriot and believed that the elite joined the military. In senior year I never applied for any scholarships and I was offered one but turned it down because I already signed the papers. I thought I set myself up for success. Now I believe I was dead wrong for joining. The only benefit I see so far after a year and a half of service is that I'm trying to set myself up financially before I get out and hopefully attend college. It sounds like a plan but I feel no happiness from what I do at all. I convinced myself there's no honor in it anymore; it's just another job. I don't exercise by myself anymore. I feel like I'm not progressing anywhere in life being in service. I'm just a body, and if I wasn't here doing what I'm doing, there'd just be somebody else doing the exact same. I'm replaceable. That's the mindset the military gave me. I look forward to going back home in 6 months for vacation, and that's the only thing I've been looking forward to since I've been stationed. After that, the only thing I have my eyes on is getting out of service, going home, being closer to my family again. There's nothing here that satisfies me, and I hate it. I feel like I've tried everything to be happy here but it seems impossible. I wish somebody could help.",
    "annotations": {
      "Feeling-bad-about-yourself-or-that-you-are-a-failure-or-have-let-yourself-or-your-family-down": [
        "I thought I set myself up for success. Now I believe I was dead wrong for joining."
      ],
      "Feeling-down-depressed-or-hopeless": [],
      "Feeling-tired-or-having-little-energy": [
        "I feel like I'm not progressing anywhere in life being in service."
      ],
      "Little-interest-or-pleasure-in-doing": [
        "There's nothing here that satisfies me, and I hate it."
      ],
      "Moving-or-speaking-so-slowly-that-other-people-could-have-noticed-Or-the-opposite-being-so-fidgety-or-restless-that-you-have-been-moving-around-a-lot-more-than-usual": [],
      "Poor-appetite-or-overeating": [],
      "Thoughts-that-you-would-be-better-off-dead-or-of-hurting-yourself-in-some-way": [],
      "Trouble-concentrating-on-things-such-as-reading-the-newspaper-or-watching-television": [],
      "Trouble-falling-or-staying-asleep-or-sleeping-too-much": []
    }
  },

May I proceed with the rest of the INPUTS? """

In [ ]:
import json

results = []

# Function to process each JSON object and collect the output
def process_and_collect_output(obj):
    # Construct the prompt
    full_prompt = f"{base_instruction} {json.dumps(obj, indent=2)}"

    # Prepare the payload
    payload = {
        "messages": [
            {"role": "user", "content": full_prompt}
        ],
        "model": "llama-3.1-8b-instant"
    }

    try:
        # Call the Groq API
        chat_completion = client.chat.completions.create(**payload)

        # Extract the LLM output
        llm_output = chat_completion.choices[0].message.content

        # Clean up the LLM output (remove unwanted characters or text if necessary)
        llm_output = llm_output.strip()  # Strip any extra whitespace
        # Handle cases where output is not valid JSON
        try:
            result = json.loads(llm_output)
        except json.JSONDecodeError:
            print(f"Error decoding JSON from output: {llm_output}")
            result = {}  # Use an empty dictionary or handle as needed

        # Append output to results list
        results.append(result)

    except Exception as e:
        print(f"Error processing object: {e}")

# Process and collect results for each object
for obj in input_data:
    process_and_collect_output(obj)

# Write results to output file with comma-separated JSON objects
with open(output_file_path, "w") as output_file:
    output_file.write("[\n")  # Start the JSON array
    first = True
    for result in results:
        if not first:
            output_file.write(",\n")
        json.dump(result, output_file, indent=2)
        first = False
    output_file.write("\n]")  # End the JSON array


Error processing object: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01j5nsmm8xfc084dq4sm82m2qx` on : Limit 1000000, Used 1000552, Requested 1990. Please try again in 3m39.6522s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error processing object: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01j5nsmm8xfc084dq4sm82m2qx` on : Limit 1000000, Used 1000551, Requested 2805. Please try again in 4m50.0092s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}
Error processing object: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01j5nsmm8xfc084dq4sm82m2qx` on : Limit 1000000, Used 1000550, Requested 1957. Please try again in 3m36.67s. Visit https://console.g